In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('clustering').getOrCreate()

In [0]:
from pyspark.ml.clustering import KMeans

In [0]:
df = spark.read.format('libsvm').load('/FileStore/tables/sample_kmeans_data.txt')

In [0]:
df.show()

+-----+--------------------+
label| features|
+-----+--------------------+
 0.0| (3,[],[])|
 1.0|(3,[0,1,2],[0.1,0...|
 2.0|(3,[0,1,2],[0.2,0...|
 3.0|(3,[0,1,2],[9.0,9...|
 4.0|(3,[0,1,2],[9.1,9...|
 5.0|(3,[0,1,2],[9.2,9...|
+-----+--------------------+

In [0]:
final_df = df.select('features')

In [0]:
final_df.show()

+--------------------+
 features|
+--------------------+
 (3,[],[])|
(3,[0,1,2],[0.1,0...|
(3,[0,1,2],[0.2,0...|
(3,[0,1,2],[9.0,9...|
(3,[0,1,2],[9.1,9...|
(3,[0,1,2],[9.2,9...|
+--------------------+

In [0]:
km = KMeans().setK(2).setSeed(1)

In [0]:
km_model = km.fit(final_df)

In [0]:
wssse = km_model.computeCost(final_df)

--------------------------------------------------------------------------- 
 AttributeError Traceback (most recent call last)
 <command-736861679484500> in <module> 
 ----> 1 wssse = km_model . computeCost ( final_df ) 

 AttributeError : 'KMeansModel' object has no attribute 'computeCost'

In [0]:
print(wssse)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-736861679484504> in <module> 
 ----> 1 print ( wssse ) 

 NameError : name 'wssse' is not defined

In [0]:
final_df.show()

+--------------------+
 features|
+--------------------+
 (3,[],[])|
(3,[0,1,2],[0.1,0...|
(3,[0,1,2],[0.2,0...|
(3,[0,1,2],[9.0,9...|
(3,[0,1,2],[9.1,9...|
(3,[0,1,2],[9.2,9...|
+--------------------+

In [0]:
centers = km_model.clusterCenters()

In [0]:
centers

Out[19]: [array([9.1, 9.1, 9.1]), array([0.1, 0.1, 0.1])]

In [0]:
predictions = km_model.transform(final_df)

In [0]:
predictions.show()

+--------------------+----------+
 features|prediction|
+--------------------+----------+
 (3,[],[])| 1|
(3,[0,1,2],[0.1,0...| 1|
(3,[0,1,2],[0.2,0...| 1|
(3,[0,1,2],[9.0,9...| 0|
(3,[0,1,2],[9.1,9...| 0|
(3,[0,1,2],[9.2,9...| 0|
+--------------------+----------+

In [0]:
km2 = KMeans().setK(3).setSeed(1)

In [0]:
km_model2 = km2.fit(final_df)

In [0]:
centers2 = km_model2.clusterCenters()
centers2

Out[26]: [array([9.1, 9.1, 9.1]), array([0.05, 0.05, 0.05]), array([0.2, 0.2, 0.2])]

In [0]:
predictions2 = km_model2.transform(final_df)

In [0]:
predictions2.show()

+--------------------+----------+
 features|prediction|
+--------------------+----------+
 (3,[],[])| 1|
(3,[0,1,2],[0.1,0...| 1|
(3,[0,1,2],[0.2,0...| 2|
(3,[0,1,2],[9.0,9...| 0|
(3,[0,1,2],[9.1,9...| 0|
(3,[0,1,2],[9.2,9...| 0|
+--------------------+----------+

In [0]:
spark = SparkSession.builder.appName('clusteringseeds').getOrCreate()

In [0]:
 df = spark.read.csv('/FileStore/tables/seeds_dataset.csv',inferSchema=True,header=True)

In [0]:
df.printSchema()

root
-- area: double (nullable = true)
-- perimeter: double (nullable = true)
-- compactness: double (nullable = true)
-- length_of_kernel: double (nullable = true)
-- width_of_kernel: double (nullable = true)
-- asymmetry_coefficient: double (nullable = true)
-- length_of_groove: double (nullable = true)

In [0]:
df.head(1)

Out[32]: [Row(area=15.26, perimeter=14.84, compactness=0.871, length_of_kernel=5.763, width_of_kernel=3.312, asymmetry_coefficient=2.221, length_of_groove=5.22)]

In [0]:
from pyspark.ml.feature import VectorAssembler

In [0]:
df.columns

Out[34]: ['area',
 'perimeter',
 'compactness',
 'length_of_kernel',
 'width_of_kernel',
 'asymmetry_coefficient',
 'length_of_groove']

In [0]:
assembler = VectorAssembler(inputCols=df.columns,outputCol='features')

In [0]:
final_df = assembler.transform(df)

In [0]:
final_df.printSchema()

root
-- area: double (nullable = true)
-- perimeter: double (nullable = true)
-- compactness: double (nullable = true)
-- length_of_kernel: double (nullable = true)
-- width_of_kernel: double (nullable = true)
-- asymmetry_coefficient: double (nullable = true)
-- length_of_groove: double (nullable = true)
-- features: vector (nullable = true)

In [0]:
from pyspark.ml.feature import StandardScaler

In [0]:
scaler = StandardScaler(inputCol='features',outputCol='features_scaled')

In [0]:
scaler_model = scaler.fit(final_df)

In [0]:
df_transform = scaler_model.transform(final_df)

In [0]:
df_transform.head(1)

Out[52]: [Row(area=15.26, perimeter=14.84, compactness=0.871, length_of_kernel=5.763, width_of_kernel=3.312, asymmetry_coefficient=2.221, length_of_groove=5.22, features=DenseVector([15.26, 14.84, 0.871, 5.763, 3.312, 2.221, 5.22]), features_scaled=DenseVector([5.2445, 11.3633, 36.8608, 13.0072, 8.7685, 1.4772, 10.621]))]

In [0]:
kmeans = KMeans(featuresCol='features_scaled',k=3)

In [0]:
kmeans_model = kmeans.fit(df_transform)

In [0]:
print('WSSSE')
print(kmeans_model.computeCost(df_transform))

WSSSE

--------------------------------------------------------------------------- 
 AttributeError Traceback (most recent call last)
 <command-2869066399632900> in <module> 
 1 print ( 'WSSSE' ) 
 ----> 2 print ( kmeans_model . computeCost ( df_transform ) ) 

 AttributeError : 'KMeansModel' object has no attribute 'computeCost'

In [0]:
centers = kmeans_model.clusterCenters()

In [0]:
print(centers)

[array([ 4.93382436, 10.94691274, 37.30542404, 12.41332714, 8.60366812,
 1.82917353, 10.40106154]), array([ 6.35645488, 12.40730852, 37.41990178, 13.93860446, 9.7892399 ,
 2.41585013, 12.29286107]), array([ 4.06660859, 10.14191893, 35.84098009, 11.81592066, 7.52397236,
 3.1823335 , 10.39801233])]

In [0]:
kmeans_model.transform(df_transform).select('prediction').show()

+----------+
prediction|
+----------+
 0|
 0|
 0|
 0|
 0|
 0|
 0|
 0|
 1|
 0|
 0|
 0|
 0|
 0|
 0|
 0|
 0|
 0|
 0|
 2|
+----------+
only showing top 20 rows